<a href="https://colab.research.google.com/github/Soroosh-N/Deep-learning-basics/blob/main/HW1_soroosh_noorzad_99205372.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep learning assignment - 1st series<br/>**
Soroosh Noorzad - 99205372<br/>
soroosh.noorzad@gmail.com<br/>
Phone : +989111853686<br/>
Date : 1400.08

# **READ ME FIRST :**
To See the results of **Question 1 parts**, please run the cells one by one untill reaching the answering sections.<br/>
To See the results of **Question 2 parts**, please scroll down to the Q2 section, then run its cells.<br/>

---

Note that, we will have our necessary descriptions on the report of the assignment.

---

**Contents of the notebook :**
*   Libraries (Q1)
*   Functions (Q1)
*   Datasets (Q1)
*   Answer to section Q1.1
*   Answer to section Q1.2-3-4-5-6
*   Answer to section Q1.7
*   Answer to section Q2



# **Libraries :**
This is the place where all the libraries of the question one will be gathered.

In [1]:
# We should use tensorflow 1 in our project, so:
%tensorflow_version 1.x
import tensorflow as tf
print('TensorFlow version :', tf.__version__)

# Other libraries
import cv2
import math
import time
import itertools
import numpy as np
import pandas as pd
from threading import Thread
from matplotlib import pyplot as plt
from PIL import ImageDraw, Image, ImageFont

TensorFlow 1.x selected.
1.15.2


# **Functions :**
All the general functions which can be used all over the project will be here to prevenet repition.

In [4]:
def csv_file_to_numpy_array(csv_file):
    """
    Convert this special dataset csv files to numpy arrays
    :param csv_file:        input csv file
    :return:                inputs and their corresponding labels
    """
    x_df = csv_file.drop(csv_file.columns[0], axis=1).div(255).to_numpy()
    y_df = csv_file.drop(csv_file.columns[1:], axis=1)
    XX = []
    for row in x_df:
        XX.append(np.reshape(row, (28, 28)))
    return np.array(XX), y_df[0].to_numpy()


def line_def(string="-", line_length=60):
    """
    print a line on the output
    :param string:          the character line made of
    :param line_length:     the length of line
    :return:                the line to be printed
    """
    return ''.join(string for _ in range(line_length))


def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap='Oranges'):
    """
    This function prints and plots the confusion matrix.
    :param cm:              Confusion Matrix
    :param classes:         Labels to be printed on the plot
    :param normalize:       Normalization can be applied by setting `normalize=True`.
    :param title:           The plot title
    :param cmap:            Color of the plot
    :return:                nothing
    """
    plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Labels')
    plt.xlabel('Predictions')
    plt.subplots_adjust(left=0.1, right=1, top=0.95, bottom=0.05)


def bar_plot(input_vector, input_labels, input_texts):
    """
    Print a bar plot from the input data
    :param input_vector:        the input data
    :param input_labels:        the classes to be counted
    :param input_texts:         the titles and labels and other stuff of the plot
    :return:                    nothing
    """
    fig = plt.figure()
    t = [input_labels[i] for i in range(len(input_labels))]
    count = [0 for _ in range(len(input_labels))]
    for i, _ in enumerate(input_vector):
        count[input_vector[i]] += 1
    ax = fig.add_subplot(1, 1, 1)
    ax.bar(t, count, color="#4CAF50", width=0.3)
    ax.set_title(input_texts[0])
    ax.set_ylabel(input_texts[1])
    ax.set_xlabel(input_texts[2])
    # ax.legend(loc="best")
    plt.show()


def plot_2d(dimension, docs, label, legends, colors, plot_type="plot", marker="."):
    """
    To plot 2d figures (With the use of matplotlib)
    ----------
    Parameters:
        dimension (list): A list of all the sections of one figure, e.g. in 2x2 figs we have 221, 222, 223, 224 dims.
        docs (list): This gonna be tricky. :) We included all the plots on this variable in such a format that we can
        use this function under different situations.
        label (list): A complex list of labels for all the plots in each figure.
        legends (list): A list of corresponding legends to describe every curve in the plots.
        colors (list): A list of corresponding colors for every curve we see in the plots.
        plot_type (str): Should we sketch the main curve by scattering or plotting? or maybe histogram?
        marker (str): The main plot marker.
    ----------
    Returns:
        (Object) The function will return the created figure (by the use of matplotlib).
    """
    fig = plt.figure()
    for i, d in enumerate(docs):
        ax = fig.add_subplot(dimension[i])
        t1 = np.arange(0, len(d[0]), 1)
        for j in range(len(d)):
            if plot_type == "scatter":
                ax.scatter(t1, d[j], color=(colors[j]), marker=marker, label=legends[j])
                ax.set_ylim(bottom=-1.1, top=1.1)
            else:
                # simple plotting
                ax.plot(t1, d[j], color=(colors[j]), label=legends[j])
        ax.set_title(label[0][i])
        ax.set_xlabel(label[1])
        ax.set_ylabel(label[2])
        ax.grid(which='both')
        ax.legend(loc="best")
        # manager = plt.get_current_fig_manager()
        # manager.window.showMaximized()
    return fig


def print_res(figs, titles, ext="pdf"):
    """
    This is a function of saving final plots to the output files
    ----------
    Parameters:
        figs (list): A list of figure objects, produced with matplotlib.
        titles (list): A list of titles corresponding to the figures given above. We use these as output file names.
        ext (str): What extension(type) are the final outputs.
    ----------
    Returns:
        Nothing
    """
    # Let's sleep one second. Don't rush! The figures are not ready yet! A little latency will results better pictures!
    time.sleep(1)
    for i, fig in enumerate(figs):
        fig.savefig(titles[i] + "." + ext)


def remove_uniformly(x_data, y_data, label_position, how_much=5):
    """
    This function will remove samples from dataset and leave one of 'how_much' samples to be untouched
    :param x_data:              input data to be sampled
    :param y_data:              labels to be sampled
    :param label_position:      what label to be removed uniformly
    :param how_much:            how is the sampling rate
    :return:                    the sampled dataset
    """
    c = 0
    members_to_delete = []
    for index, y in enumerate(y_data):
        if y == label_position-1:
            c += 1
            if c == how_much:
                c = 0
            else:
                members_to_delete.append(index)
    x_data = np.delete(x_data, members_to_delete, axis=0)
    y_data = np.delete(y_data, members_to_delete)
    return x_data, y_data


def confusion_accuracy(confusion_matrix):
    """
    This function will provide accuracy of each of confusion matrix labels
    :param confusion_matrix:        The confusion matrix
    :return:                        A list of labels accuracy, and! mean of that list
    """
    labels_acc = [0 if sum(row) == 0 else round(row[index] / sum(row), 2) for index, row in enumerate(confusion_matrix)]
    mean_accuracy = round(sum(labels_acc)/len(labels_acc), 2)
    return labels_acc, mean_accuracy


# **Datasets :**
Loading dataset from google colab.

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
          'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
          'T', 'U', 'V', 'W', 'X', 'Y']


Mounted at /content/gdrive


# **Answer to section Q1.1 :**
Printing the first 25 pictures of the train dataset.

In [ ]:
csv_train = pd.read_csv('gdrive/MyDrive/golab/datasets/Q1_train.csv', sep=',', skiprows=1, header=None)
x_train, y_train = csv_file_to_numpy_array(csv_train)

for index, picture in enumerate(x_train):
    if index < 25:
        img = Image.fromarray(np.uint8(picture * 255), 'L')
        plt.subplot(5, 5, index+1)
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.subplots_adjust(bottom=0.1, top=0.9, wspace=None, hspace=1)
        plt.title("#"+str(index) + " : " + str(labels[y_train[index]]), fontsize=8)
        plt.imshow(img, cmap='gray')
    else:
        break
plt.show()


# **Answer to section Q1.2-3-4-5-6 :**
In this section we created a procedure to handle all the sections of this question. With just changing the hyper-parameters we can answer different questions.

In [ ]:
csv_train = pd.read_csv('gdrive/MyDrive/golab/datasets/Q1_train.csv', sep=',', skiprows=1, header=None)
csv_test = pd.read_csv('gdrive/MyDrive/golab/datasets/Q1_test.csv', sep=',', skiprows=1, header=None)

x_train, y_train = csv_file_to_numpy_array(csv_train)
x_test, y_test = csv_file_to_numpy_array(csv_test)

# TODO : Constants:
# dataset class count
class_count = 25

# TODO : Hyper-parameters
learning_rate = 0.1
batch_size = 128
seed_number = 35427
valid_data_percentage = 0.2
epoch_count = 150
# ADAM or not? (if False then SGD)
ADAM_opt = False
# Setting for showing results at the end :
display_step = 1
# Dropout
dropout_apply = False
dropout_rate = 0.5
# Layers setting (count, dropout, ReLU, bath_norm):
layers_neuron_count = [784, 128, 128, class_count]
dropout = [
    (dropout_apply, dropout_rate),
    (dropout_apply, dropout_rate),
    (False, 0)
]
relus = [True, True, False]
batch_norm = [False, False, False]
# Do we want to remove "THAT" char uniformly?
# [True or False, THAT_char_position]
# E character is the 5th character in labels. so position = 5
remove_char = [False, 5]


# TODO : Functions:
def fc_layer(previous_layer, layer_def, training):
    layer = tf.add(tf.matmul(previous_layer, layer_def[0][0]), layer_def[0][1])
    if layer_def[1][0]:
        layer = tf.nn.dropout(layer, keep_prob=(1-layer_def[1][1]))
    if layer_def[2]:
        layer = tf.nn.relu(layer)
    if layer_def[3]:
        layer = tf.layers.batch_normalization(layer, training=training)
    return layer


def neural_network(input_matrix, layers_def, training):
    hidden = tf.layers.flatten(input_matrix)
    for layer_def in layers_def:
        hidden = fc_layer(hidden, layer_def, training)
    return hidden


# TODO : Pre-processes:
[height, width] = x_train.shape[1:]
num_test = x_test.shape[0]
num_train = int((1-valid_data_percentage) * x_train.shape[0])
steps = math.floor(num_train/batch_size)

x_valid = x_train[num_train:]
y_valid = y_train[num_train:]

x_train = x_train[0:num_train]
y_train = y_train[0:num_train]

# Sample from "THAT!" special char, uniformly :
if remove_char[0]:
    # I made up remove_uniformly from myself. look at functions section.
    x_train, y_train = remove_uniformly(x_train, y_train, remove_char[1], 5)

print("Total records :", x_train.shape[0]+x_valid.shape[0])
print("Train records count :", x_train.shape[0])
print("Valid records count :", x_valid.shape[0])
print("Batch size :", batch_size)
print("Epoch count :", epoch_count)

# finalizing the data precesses
with tf.Session() as sess:
    y_train_hot = sess.run(tf.one_hot(y_train, class_count))
    y_valid_hot = sess.run(tf.one_hot(y_valid, class_count))
    y_test_hot = sess.run(tf.one_hot(y_test, class_count))
    x_train_shuffle = sess.run(tf.random.shuffle(x_train, seed=seed_number))
    y_train_shuffle = sess.run(tf.random.shuffle(y_train_hot, seed=seed_number))

# TODO : Make Network graph
# To clear the defined variables and operations of the previous cell
tf.reset_default_graph()
# tf Graph inputs (X for input data and Y for output labels)
T = tf.placeholder(dtype=tf.bool, name='TRAINING')
X = tf.placeholder(dtype=tf.float32, shape=(None, width, height), name='INPUT')
Y = tf.placeholder(dtype=tf.float32, shape=(None, class_count), name='OUTPUT')
# defining weights
w = [
    tf.get_variable(
        dtype=tf.float32,
        shape=(layers_neuron_count[i], layers_neuron_count[i+1]),
        name='W'+str(i+1),
        initializer=tf.truncated_normal_initializer(stddev=0.001)) for i in range(len(layers_neuron_count)-1)
]
# defining biases
b = [
    tf.get_variable(
        dtype=tf.float32,
        shape=layers_neuron_count[i+1],
        name='B'+str(i+1),
        initializer=tf.zeros_initializer()) for i in range(len(layers_neuron_count)-1)
]
layers_definition = [[[w[i], b[i]], dropout[i], relus[i], batch_norm[i]] for i in range(len(layers_neuron_count)-1)]
# create the model
logits = neural_network(X, layers_definition, T)

# _op suffix means "TensorFlow Operations", also known as "Ops"
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits, name='LOSS'))
# In relation to the batch normalization, we have line below :
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

# TODO : Setting up optimizer
if ADAM_opt:
    # ADAM optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='ADAM')
else:
    # SGD optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate, name='SGD')
train_op = optimizer.minimize(loss_op)

# TODO : Evaluate model
softmax = tf.nn.softmax(logits, name="SOFTMAX")
prediction = tf.argmax(softmax, axis=1, name="PREDICTION")
y_labels = tf.argmax(Y, 1)
correct_pred = tf.equal(prediction, y_labels)
# defining accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name="ACCURACY")
# defining confusion matrix
cm = tf.confusion_matrix(labels=y_labels, predictions=prediction)
# defining model initializer
init = tf.global_variables_initializer()

# Model saver
saver = tf.train.Saver()

# TODO : Create Summaries
# Create a summary to monitor weight tensors
w_s = [tf.summary.histogram("layer_number_" + str(i+1) + "_weights", w[i]) for i in range(len(layers_neuron_count)-1)]
# Merge some of these summaries into a single op for train dataset
train_merged_op = tf.summary.merge(w_s)

# Create a summary to monitor cost tensor
los_s = tf.summary.scalar("Loss_scalar", loss_op)
# Create a summary to monitor accuracy tensor
acc_s = tf.summary.scalar("Accuracy_scalar", accuracy)
# Merge some of these summaries into a single op for test dataset
test_merged_op = tf.summary.merge([los_s, acc_s])

# We don't want to merge them all. we have two different goals.
# merged_summary_op = tf.summary.merge_all()

# TODO : Starting a session
# Starting a time recorder
t = time.time()
with tf.Session() as sess:
    # Initialization
    sess.run(init)
    # Pre-allocation
    train_los_set, train_acc_set = [], []
    valid_los_set, valid_acc_set = [], []
    test_los_set, test_acc_set = [], []
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter("logs_path/", graph=tf.get_default_graph())
    for epoch in range(epoch_count):
        # Pre-allocation of train mean parameters
        train_loss_avr = 0
        train_accu_avr = 0
        # Display results
        num = ((epoch+1) / display_step) - int((epoch+1) / display_step)
        if num == 0:
            print(line_def("="))
            print('Epoch = %s ' % (epoch+1))
            print(line_def(), "\ntitle", "\t\t\t\tLoss", '\t\tAccuracy', end="\n"+line_def()+"\n")
        for i in range(steps):
            current_step = epoch * steps + i
            # After shuffling the train data we feed them in the model to catch new values for every one of them
            # By running train_op, we will train our model. we don't need any type of output for that.
            # By running loss_op and accuracy we will catch those parameters as the model learns
            f = {X: x_train_shuffle[i*batch_size: (i+1)*batch_size], Y: y_train_shuffle[i*batch_size: (i+1)*batch_size], T: True}
            _, _, b_loss, b_acc, train_summary = sess.run([train_op, extra_update_ops, loss_op, accuracy, train_merged_op], feed_dict=f)
            # Write logs at every iteration
            summary_writer.add_summary(train_summary, current_step)  # scalar summaries
            # Compute average loss
            train_loss_avr += b_loss / steps
            # Compute average accuracy
            train_accu_avr += b_acc / steps
        f_valid = {X: x_valid, Y: y_valid_hot, T: False}
        f_test = {X: x_test, Y: y_test_hot, T: False}
        # Calculate output parameters for valid records per epoch
        valid_loss, valid_acc = sess.run([loss_op, accuracy], feed_dict=f_valid)
        # Calculate output parameters for test records per epoch
        test_loss, test_acc, test_summary = sess.run([loss_op, accuracy, test_merged_op], feed_dict=f_test)
        # Write logs at every iteration
        summary_writer.add_summary(test_summary, epoch * steps)  # Histogram summaries
        # Gather results per epoch
        train_los_set.append(train_loss_avr)
        train_acc_set.append(train_accu_avr * 100)
        valid_los_set.append(valid_loss)
        valid_acc_set.append(valid_acc * 100)
        test_los_set.append(test_loss)
        test_acc_set.append(test_acc * 100)
        if num == 0:
            # Display results
            print("Training", "\t\t\t{:.4f}".format(train_loss_avr), "\t\t{:.1f}".format(train_accu_avr * 100), "%")
            # print(line_def("."))
            print('Validation', '\t\t\t{:.4f}'.format(valid_loss), "\t\t{:.1f}".format(valid_acc*100), "%")
            # print(line_def("."))
            print("Testing\t\t\t\t{:.4f}".format(test_loss), '\t\t{:.1f}'.format(test_acc * 100), "%")
            # print(line_def(), end="\n\n")
            print('Time passed :', int(time.time() - t))
    # Evaluating confusion matrix
    confusion_matrix = sess.run(cm, feed_dict={X: x_test, Y: y_test_hot, T: False})
    # Save model weights to disk
    save_path = saver.save(sess, "saved_model")
    print("Model saved in file: %s" % save_path)

# TODO : Ending time recorder
print('\nProcessing time :', int(time.time() - t))
labels_acc, mean_acc = confusion_accuracy(confusion_matrix)
print("The most accurate label is", labels[np.argmax(labels_acc)], "with a percent of {:.1f}".format(max(labels_acc)))
print("The entire labels accuracy list :", labels_acc)
print("The mean of labels accuracy is {:.1f}%".format(mean_acc * 100))

# TODO : plot confusion matrix
plot_confusion_matrix(confusion_matrix, labels)

# TODO : plot labels scattering
bar_plot(y_train, labels, ["Train data scattering", 'Repetition', 'Labels'])
bar_plot(y_valid, labels, ["Valid data scattering", 'Repetition', 'Labels'])
bar_plot(y_test, labels, ["Test data scattering", 'Repetition', 'Labels'])

# TODO : plot loss and accuracy curves
los_labels = [["Loss Plots"], 'Epochs', 'Loss value']
acc_labels = [["Accuracy Plots"], 'Epochs', 'Accuracy %']
colors = ['black', 'blue', 'brown']
legends = ['Train data', 'Valid data', 'Test data']
los_fig = plot_2d([111], [[train_los_set, valid_los_set, test_los_set]], los_labels, legends, colors)
acc_fig = plot_2d([111], [[train_acc_set, valid_acc_set, test_acc_set]], acc_labels, legends, colors)
figs = [los_fig, acc_fig]
titles = ["[00] Los_fig", '[00] Acc_fig']
thread = Thread(target=print_res, args=(figs, titles, "jpg"))
thread.start()
plt.show()

# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')


# **Answer to Section Q1.7**
In this part, we will get the saved model from the previous part and run the camera using this model to detect the hand gestures.
Attention :
*    Before running, please make sure to run every previous cells. This cell may be dependent on their results.
*    To stop the video stream, just click on the video.

In [ ]:
# import dependencies
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html


# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    """
    Params:
          js_reply: JavaScript object containing image from webcam
    Returns:
          img: OpenCV BGR image
    """
    # decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img


# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
        bytes: Base64 image byte string
    """
    # convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
      stream.getVideoTracks()[0].stop();
      video.remove();
      div.remove();
      video = null;
      div = null;
      stream = null;
      imgElement = null;
      captureCanvas = null;
      labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
          
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
  display(js)


# dataset class count
class_count = 25
valid_data_percentage = 0.2

# delete the current graph
tf.reset_default_graph()

# import the graph from the file
imported_graph = tf.train.import_meta_graph('saved_model.meta')

# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 

# run the session
with tf.Session() as sess:
    # restore the saved neural network model
    imported_graph.restore(sess, 'saved_model')
    # start streaming video from webcam
    video_stream()
    while True:
        js_reply = eval_js('stream_frame("{}", "{}")'.format(label_html, bbox))
        if not js_reply:
            break
        # convert JS response to OpenCV Image
        frame = js_to_image(js_reply["img"])
        # create transparent overlay for bounding box
        bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (28, 28)) / 255
        prediction, softmax = sess.run(["PREDICTION:0", "SOFTMAX:0"], feed_dict={"INPUT:0": [resize], "OUTPUT:0": [[0 for _ in range(class_count)]]})
        text = labels[prediction[0]] + " [" + "{:.2f}".format(softmax[0][prediction[0]] * 100) + " %]"
        cv2.putText(
            bbox_array,                      # Frame to put
            text,                       # Text to show
            (10, 40),                   # Position
            cv2.FONT_HERSHEY_SIMPLEX,   # Font family
            1,                          # Font size
            (0, 0, 255),                # Font color
            2,                          # Font Stroke
            cv2.LINE_4                  # ?
        )
        bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
        bbox = bbox_to_bytes(bbox_array)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


# **Answer to Section Q2 :**
In this section, we added missing lines to the codes. After running the codes (one after each other) we will let the results to be saved on the notebook for viewers to see how the codes will ended up.<br/>
We will describe what happened here on the report of assignment.

In [ ]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash

    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/grading.py -O ../grading.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week1_intro/submit.py

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# if you see "<classname> has no attribute .env", remove .env or update gym
env = gym.make("CartPole-v0").env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]

plt.imshow(env.render("rgb_array"))
print("state vector dim =", state_dim)
print("n_actions =", n_actions)

In [ ]:
from sklearn.neural_network import MLPClassifier

agent = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation='tanh',
)

# initialize agent to the dimension of state space and number of actions
agent.partial_fit([env.reset()] * n_actions, range(n_actions), range(n_actions))

*    `agent.partial_fit(states, actions)` - make a single training pass over the data. Maximize the probabilitity of :actions: from :states:
*    `agent.predict_proba(states)` - predict probabilities of all actions, a matrix of shape **[len(states), n_actions]**

In [ ]:
def generate_session(env, agent, t_max=1000):
    """
    Play a single game using agent neural network.
    Terminate when game finishes or after :t_max: steps
    """
    states, actions = [], []
    total_reward = 0

    s = env.reset()

    for t in range(t_max):
        
        # use agent to predict a vector of action probabilities for state :s:
        probs = agent.predict_proba([s])[0]

        assert probs.shape == (env.action_space.n,), "make sure probabilities are a vector (hint: np.reshape)"
        
        # use the probabilities you predicted to pick an action
        # sample proportionally to the probabilities, don't just take the most likely action
        a = np.random.choice(range(n_actions), size=1, p=probs)[0]
        # ^-- hint: try np.random.choice

        new_s, r, done, info = env.step(a)

        # record sessions like you did before
        states.append(s)
        actions.append(a)
        total_reward += r
        s = new_s
        if done:
            break
    return states, actions, total_reward

In [ ]:
dummy_states, dummy_actions, dummy_reward = generate_session(env, agent, t_max=5)
print("states:", np.stack(dummy_states))
print("actions:", dummy_actions)
print("reward:", dummy_reward)

CEM steps

Deep CEM uses exactly the same strategy as the regular CEM.

The only difference is that now each observation is not a number but a `float32` vector.

In [ ]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    Please return elite states and actions in their original order 
    [i.e. sorted by session number and timestep within session]

    """

    reward_threshold = np.percentile(rewards_batch, percentile)

    elite_states = [s for i in range(len(states_batch)) if rewards_batch[i]>=reward_threshold for s in states_batch[i]]
    elite_actions = [a for i in range(len(actions_batch)) if rewards_batch[i]>=reward_threshold for a in actions_batch[i]]
    
    return elite_states, elite_actions

In [ ]:
from IPython.display import clear_output

def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    A convenience function that displays training progress. 
    No cool math here, just charts.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])

    clear_output(True)
    print("mean reward = %.3f, threshold=%.3f" % (mean_reward, threshold))
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.plot(list(zip(*log))[1], label='Reward thresholds')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines([np.percentile(rewards_batch, percentile)], [0], [100], label="percentile", color='red')
    plt.legend()
    plt.grid()

    plt.show()

In [ ]:
n_sessions = 100
percentile = 80
log = []

for i in range(100):
    # generate new sessions
    sessions = [ generate_session(env, agent) for _ in range(n_sessions) ]

    states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))

    # elite_states, elite_actions = map(np.concatenate,[states_batch,actions_batch])
    elite_states, elite_actions = select_elites(states_batch,actions_batch,rewards_batch,percentile)

    agent.fit(elite_states, elite_actions)
    # <YOUR CODE: partial_fit agent to predict elite_actions(y) from elite_states(X)>

    show_progress(rewards_batch, log, percentile, reward_range=[0, np.max(rewards_batch)])

    if np.mean(rewards_batch) > 190:
        print("You Win!")


Results :

In [ ]:
# Record sessions

import gym.wrappers

with gym.wrappers.Monitor(gym.make("CartPole-v0"), directory="videos", force=True) as env_monitor:
    sessions = [generate_session(env_monitor, agent) for _ in range(100)]

In [ ]:
# Show video. This may not work in some setups. If it doesn't
# work for you, you can download the videos and view them locally.

from pathlib import Path
from IPython.display import HTML

video_names = sorted([s for s in Path('videos').iterdir() if s.suffix == '.mp4'])

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_names[-1]))  # You can also try other indices